# <center><font size=12 color="Brown">ODI Cricket Statistics</font></center>

<font size=6>Code by <b>Neelmani</b></font>

In [1]:
import pandas as pd

In [2]:
from bokeh.io import curdoc,output_notebook
from bokeh.layouts import column,row
from bokeh.models import CustomJS,ColumnDataSource,Slider,TextInput,Select
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv("Cricket.csv")
df = df.rename(columns={'100':'Hundred','50':'Fifty','0':'Ducks'})
df.head()

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,Hundred,Fifty,Ducks,4s,6s
0,SR Tendulkar�(INDIA),1989-2012,463,452,41,18426,200*,44.83,21367,86.23,49,96,20,2016,195
1,KC Sangakkara�(Asia/ICC/SL),2000-2015,404,380,41,14234,169,41.98,18048,78.86,25,93,15,1385,88
2,RT Ponting�(AUS/ICC),1995-2012,375,365,39,13704,164,42.03,17046,80.39,30,82,20,1231,162
3,ST Jayasuriya�(Asia/SL),1989-2011,445,433,18,13430,189,32.36,14725,91.20,28,68,34,1500,270
4,DPMD Jayawardene�(Asia/SL),1998-2015,448,418,39,12650,144,33.37,16020,78.96,19,77,28,1119,76


In [4]:
axis_map = {
    "Player":"Player",
    "Span":"Span",
    "Mat":"Mat",
    "Inns":"Inns",
    "Runs":"Runs",
    "HS":"HS",
    "BF":"BF",
    "SR":"SR",
    "Hundred":"Hundred",
    "Fifty":"Fifty",
    "4s":"4s",
    "6s":"6s",
}

In [5]:
runs=Slider(title="Minimum Runs Scored",value=7000,start=5000,end=18500,step=100)
match=Slider(title="Minimum Number of match played",value=170,start=130,end=470,step=10)
Inning=Slider(title="Minimum Number of Inning played",value=140,start=110,end=460,step=10)
hundred=Slider(title="Minimum Number of 100 scored",value=15,start=0,end=50,step=1)
fifty=Slider(title="Minimum Number of 50 scored",value=30,start=20,end=100,step=2)
player_name=TextInput(title="Player Name")
x_axis=Select(title="X-Axis",options=sorted(axis_map.keys()),value="Mat")
y_axis=Select(title="Y-Axis",options=sorted(axis_map.keys()),value="Runs")

In [6]:
source=ColumnDataSource(data=dict(x=[],y=[],player=[],span=[],ave=[]))

In [7]:
TOOLTIPS=[
    ("Player","@player"),
    ("Span","@span"),
    ("Ave","@ave")
]

In [8]:
p=figure(plot_height=600,plot_width=600,toolbar_location='above',tooltips=TOOLTIPS)
p.circle(x="x",y="y",source=source,size=7)

GlyphRenderer(id='1048', ...)

In [9]:
def select_runs():
    player=player_name.value.strip()
    selects=df[
        (df.Runs >= runs.value) &
        (df.Mat >= match.value) &
        (df.Inns >= Inning.value) &
        (df.Hundred >= hundred.value) &
        (df.Fifty >= fifty.value)
    ]
    if(player != ""):
        selects=selects[selects.Player.str.contains(player)==True]
    return selects

In [10]:
def update():
    
    df1=select_runs()
    x_name=axis_map[x_axis.value]
    y_name=axis_map[y_axis.value]
    
    p.xaxis.axis_label=x_axis.value
    p.yaxis.axis_label=y_axis.value
    source.data=dict(
        x=df1[x_name],
        y=df1[y_name],
        player=df1["Player"],
        span=df1["Span"],
        ave=df1["Ave"]
    )

In [11]:
controls=[runs,match,Inning,player_name,hundred,fifty,x_axis,y_axis]
for control in controls:
    control.on_change('value',lambda attr,old,new:update())

In [12]:
m=column(*controls,width=400,height=900)
l=row(m,p)
curdoc().add_root(l)
curdoc.title="ODI_Stats"
update()